script to use the dalitz plot method outlined by matt shepard to calculate the efficiency of the kstar rejection in the KKPi phasespace

The idea would be to correct the data by this as opposed to the phasespace acceptance, as this just handles the K* rejection and not the overall efficiency (which may not actually model the f1 well, unlike the signal MC)

In [1]:
import ROOT
import numpy as np
import pandas as pd
from common_analysis_tools import get_integrated_data_hist

Welcome to JupyROOT 6.24/04


In [2]:
mass_low = 1.1 # GeV
mass_high = 2.5
step_size = 0.01


kstar_mass_upper = 1.0
kstar_mass_lower = 0.8

kaon_mass = 0.493677
kshort_mass = 0.497611
pion_mass = 0.139570


In [3]:
def check_if_kstar_region(mass):
    return mass < kstar_mass_upper and mass > kstar_mass_lower


def get_weight_per_mass(mass_bin, draw_plots=False):
    phasespace_centermass = ROOT.TLorentzVector(0, 0, 0, mass_bin)
    phasespace_generator = ROOT.TGenPhaseSpace()
    phasespace_generator.SetDecay(phasespace_centermass, 3, np.array([kaon_mass, kshort_mass, pion_mass]))

    total = 0
    passed = 0

    full_dalitz = ROOT.TH2D(f"dalitz_{mass_bin}", f"dalitz_{mass_bin}", 100, 0, 2, 100, 0, 2)
    passed_dalitz = ROOT.TH2D(f"passed_dalitz_{mass_bin}", f"passed_dalitz_{mass_bin}", 100, 0, 2, 100, 0, 2)

    for _ in range(100000):
        weight = phasespace_generator.Generate()
        kaon = phasespace_generator.GetDecay(0)
        kshort = phasespace_generator.GetDecay(1)
        pion = phasespace_generator.GetDecay(2)

        neutral_kstar_mass = (kaon + pion).M()
        charged_kstar_mass = (kshort + pion).M()

        full_dalitz.Fill(neutral_kstar_mass*neutral_kstar_mass, charged_kstar_mass*charged_kstar_mass, weight)
        total += weight

        if check_if_kstar_region(neutral_kstar_mass) or check_if_kstar_region(charged_kstar_mass):
            continue 

        passed_dalitz.Fill(neutral_kstar_mass*neutral_kstar_mass, charged_kstar_mass*charged_kstar_mass, weight)
        passed += weight

    if draw_plots:
        c = ROOT.TCanvas()
        c.Divide(2,1)
        c.cd(1)
        full_dalitz.Draw("colz")
        c.cd(2)
        passed_dalitz.Draw("colz")
        c.Update()
        c.SaveAs("/work/halld/home/viducic/plots/ps_dalitz/dalitz_{}.png".format(mass_bin))
        c.Close()


    return passed / total

In [4]:
mass_bin_centers = []
kstar_cut_efficiencies = []
for mass_bin in np.arange(mass_low, mass_high+step_size, step_size):
    mass_bin_center = round(mass_bin + (step_size/2), 3)
    binned_weight = get_weight_per_mass(mass_bin_center)
    mass_bin_centers.append(mass_bin_center)
    kstar_cut_efficiencies.append(binned_weight)
    print(mass_bin_center, binned_weight)

1.105 nan
1.115 nan
1.125 nan
1.135 1.0
1.145 1.0
1.155 1.0
1.165 1.0
1.175 1.0
1.185 1.0
1.195 1.0
1.205 1.0
1.215 1.0
1.225 1.0
1.235 1.0
1.245 1.0
1.255 1.0
1.265 1.0
1.275 1.0
1.285 1.0
1.295 0.9986010202835751
1.305 0.955384100633009
1.315 0.8809088337536342
1.325 0.7961625986684868
1.335 0.715485984850758
1.345 0.6515070539649082
1.355 0.587882491239232
1.365 0.5314372160051363
1.375 0.48421293467306703
1.385 0.43998886315191493
1.395 0.40256396970897756
1.405 0.3695485908254764
1.415 0.3384451706712966
1.425 0.31218907548442576
1.435 0.284673688913585
1.445 0.2645362463387183
1.455 0.24193506933086623
1.465 0.22489390648570337
1.475 0.20848762966720522
1.485 0.19123672230706995
1.495 0.17736151639523118
1.505 0.17112138469960625
1.515 0.167965763724758
1.525 0.16790454826497547
1.535 0.16986137324493591
1.545 0.16993302846374161
1.555 0.16980821244813382
1.565 0.17097225223983126
1.575 0.17596174414735732
1.585 0.17707859962547562
1.595 0.18184120382489075
1.605 0.18811068761919

In [5]:
df = pd.DataFrame({"mass_bin_center": mass_bin_centers, "kstar_cut_efficiency": kstar_cut_efficiencies})
df.to_csv("/work/halld/home/viducic/data/ps_dalitz/kstar_cut_efficiency.csv", index=False)

In [6]:
def apply_efficiency_correction(hist):
    corrected_hist = hist.Clone()
    for i in range(hist.GetNbinsX()):
        mass_bin_center = hist.GetBinCenter(i)
        efficiency = df[df["mass_bin_center"] == round(mass_bin_center, 3)]["kstar_cut_efficiency"].values[0]
        corrected_hist.SetBinContent(i, hist.GetBinContent(i) / efficiency)
    
    corrected_hist.SetDirectory(0)

    return corrected_hist

In [11]:
hist = get_integrated_data_hist('pipkmks', 'spring', "all")
hist.Draw()

filepath: /work/halld/home/viducic/data/pipkmks/data/bestX2/pipkmks_flat_result_2018_spring.root
